In [25]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Deifne the function to draw epipolar lines
def drawlinesGray(img1,img2,lines,pts1,pts2):
    ''' img1 - img2上の点に対応するエピポーラ線を描画する画像
        lines - 対応するエピポーラ線 '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,ch = img1.shape
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

### 私のiphone12 proのカメラ内部パラメータ行列
```
カメラ内部パラメタ：
[[785.44715026   0.         505.38010871]
 [  0.         784.61050963 374.79914586]
 [  0.           0.           1.        ]]

 カメラ変形係数:
[[ 2.06842271e-01 -9.41496462e-01 -5.80054635e-04 -1.29098447e-03
   1.53535374e+00]]
```

In [26]:
img1 = cv2.imread('data/IMG_8297.jpg')  #left image
img2 = cv2.imread('data/IMG_8298.jpg')  #right image
img1 = cv2.resize(img1, ((int)(img1.shape[1]/4), (int)(img1.shape[0]/4)))
# resize img2 to match img1
h,w, c = img1.shape
img2= cv2.resize(img2, (w, h), interpolation=cv2.INTER_LINEAR)

In [27]:
# カメラ行列と歪み行列
K = np.array([
 [785.44715026,   0.,        505.38010871],
 [  0. ,        784.61050963, 374.79914586],
 [  0. ,          0.,           1.        ]])

dist_coef = np.array(
  [ 2.06842271e-01, -9.41496462e-01, -5.80054635e-04, -1.29098447e-03,1.53535374e+00])

# 画像の歪みを取る
img1 = cv2.undistort(img1, K, dist_coef, None)
img2 = cv2.undistort(img2, K, dist_coef, None)

In [28]:
# 特徴点検出と記述子の計算
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

#可視化特徴点
img1_kp = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_kp = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img12_kp = np.hstack((img1_kp, img2_kp))
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.imshow('Image', img12_kp)
cv2.waitKey(0)

-1

In [29]:
good = []
pts1 = []
pts2 = []

# マッチングのため、BFMatcher オブジェクトを作成する
bf = cv2.BFMatcher()
matches = bf.match(des1,des2)

# 距離の近い順に並べる.
matches = sorted(matches, key = lambda x:x.distance)

count = 0
for m in matches:
    count+=1
    if count < 60:
        good.append([m])
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)   
        
        
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)

# 外れ値を取り除く
pts1_inliers = pts1[mask.ravel()==1]
pts2_inliers = pts2[mask.ravel()==1]

pts1 = pts1_inliers[0:200,:]
pts2 = pts2_inliers[0:200,:]

# 右画像(二番目の画像)中の点に対応するエピポーラ線の計算
# 計算したエピポーラ線を左画像に描画
lines1 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

# 左画像(一番目の画像)中の点に対応するエピポーラ線の計算
# 計算したエピポーラ線を右画像に描画
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts1,pts2)

# 結果の表示
img35 = np.hstack((img3,img5))
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.imshow('Image', img35)
cv2.waitKey(0)

-1

In [ ]:
# まずは画像点から逆透視投影算
pts1_norm = cv2.undistortPoints(np.expand_dims(np.float32(pts1), axis=1), cameraMatrix=K, distCoeffs=None)
pts2_norm = cv2.undistortPoints(np.expand_dims(np.float32(pts2), axis=1), cameraMatrix=K, distCoeffs=None)

# 同次座標のxハットから基本行列の計算
E, mask = cv2.findEssentialMat(pts1_norm, pts2_norm, focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=3.0)
# EからR, tを計算、
# ここでRは回転行列、tは並進ベクトル
# maskは外れ値を除去するためのマスク
points, R, t, mask = cv2.recoverPose(E, pts1_norm, pts2_norm)

#座標点から3次元点を計算
M_r = np.hstack((R, t))
M_l = np.hstack((np.eye(3, 3), np.zeros((3, 1))))
P_l = np.dot(K,  M_l)
P_r = np.dot(K,  M_r)
point_4d_hom = cv2.triangulatePoints(P_l, P_r, np.expand_dims(pts1, axis=1), np.expand_dims(pts2, axis=1))
point_4d = point_4d_hom / np.tile(point_4d_hom[-1, :], (4, 1))
point_4d = point_4d[:3, :].T

# 基本行列から基礎行列計算 
Kinv = np.linalg.inv(K)
Kinvt = np.transpose(Kinv)
F = np.dot(Kinvt,E,K)

In [ ]:
# 右画像(二番目の画像)中の点に対応するエピポーラ線の計算
# 計算したエピポーラ線を左画像に描画
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img15,img16 = drawlines(img1,img2,lines1,pts1,pts2)

# 左画像(一番目の画像)中の点に対応するエピポーラ線の計算
# 計算したエピポーラ線を右画像に描画
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img13,img14 = drawlines(img2,img1,lines2,pts2,pts1)

# 結果の表示
# 結果の表示
img1315 = np.hstack((img13,img15))
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.imshow('Image', img1315)
cv2.waitKey(0)